In [1]:
from pyspark.sql.functions import col, to_date
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 3, Finished, Available, Finished)

In [14]:
not_submitted_001 = spark.read.table("Silver.dbo.silver_submission_status_view") \
    .filter((col("assignment_id") == "A001")  & (col("submission_status") == "Not submitted"))

StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 16, Finished, Available, Finished)

In [15]:
not_submitted_001.write.mode("overwrite").saveAsTable("Gold.dbo.gold_not_submitted_assignment_001")

StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 17, Finished, Available, Finished)

In [17]:
assignment_course = spark.read.table("Silver.dbo.silver_assignment_course_view")
submissions = spark.read.table("Silver.dbo.silver_student_submission_view")
students = spark.read.table("Bronze.dbo.students")

# Total students
total_students = students.count()

# Total expected submissions = students × assignments per course
expected_submissions = assignment_course.groupBy("course_id").count() \
    .withColumnRenamed("count", "assignments_per_course") \
    .withColumn("expected_submissions", col("assignments_per_course") * total_students)

# Actual submissions per course
actual_submissions = submissions.join(assignment_course, "assignment_id") \
    .groupBy("course_id").count().withColumnRenamed("count", "actual_submissions")

# Submission rate per course
submission_rate = expected_submissions.join(actual_submissions, "course_id") \
    .withColumn("submission_rate_percent", (col("actual_submissions") / col("expected_submissions")) * 100)



StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 19, Finished, Available, Finished)

In [20]:
submission_rate.write.mode("overwrite").saveAsTable("Gold.dbo.gold_submission_rate_per_course")

StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 22, Finished, Available, Finished)

In [21]:
late_submissions = spark.read.table("Silver.dbo.silver_submission_status_view") \
    .filter(col("submission_status") == "Late") \
    .groupBy("student_id") \
    .count().withColumnRenamed("count", "overdue_assignment_count")



StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 23, Finished, Available, Finished)

In [23]:
late_submissions.write.mode("overwrite").saveAsTable("Gold.dbo.gold_overdue_assignments_per_student")

StatementMeta(, 66cff26b-d410-4824-a248-0aab7356a275, 25, Finished, Available, Finished)